<a href="https://colab.research.google.com/github/hurtadosanti/TrafficSignClassifier/blob/tensorflow2/TrafficSignClassifierTf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Sign Recognition Classifier
Deep Learning

[Problem Definition](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/Traffic_Sign_Classifier.ipynb)

https://www.tensorflow.org/tutorials/keras/classification#evaluate_accuracy

## Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
import pickle
import os
from zipfile import ZipFile
from sklearn.utils import shuffle
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
print(tf.__version__)
%matplotlib inline


2.3.0


## Load Data

In [2]:
traffic_signs_file = 'traffic-signs-data.zip'
!wget --no-check-certificate --progress=bar:force:noscroll \
    https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip \
    -O traffic-signs-data.zip

!wget --no-check-certificate --progress=bar:force:noscroll \
    https://raw.githubusercontent.com/udacity/CarND-Traffic-Sign-Classifier-Project/master/signnames.csv -O signnames.csv


--2020-10-01 16:09:26--  https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.120.104
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.120.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  88.9MB/s    in 1.3s    

2020-10-01 16:09:27 (88.9 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]

--2020-10-01 16:09:27--  https://raw.githubusercontent.com/udacity/CarND-Traffic-Sign-Classifier-Project/master/signnames.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 999 [text

In [3]:
with ZipFile(traffic_signs_file, 'r') as zipObj:
   zipObj.extractall()
   print('Finish extracting')

Finish extracting


In [4]:
train = pickle.load(open("train.p", 'rb'))
valid=pickle.load(open("valid.p", 'rb'))
test = pickle.load(open("test.p", 'rb'))
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']
# Verify that the data is complete
assert(len(X_train)==len(y_train))
assert(len(X_valid)==len(y_valid))
assert(len(X_test)==len(y_test))

assert(len(X_train)>0)
assert(len(X_valid)>0)
assert(len(X_test)>0)
print('Data is correct')

Data is correct


## Data preprocessing

In [29]:
X_train_normalized = X_train/255.0
X_test_normalized = X_test/255.0
X_valid_normalized = X_valid/255.0

X_train_gray = np.sum(X_train_normalized/3,axis=3,keepdims=True)
X_test_gray = np.sum(X_test_normalized/3,axis=3,keepdims=True)
X_valid_gray = np.sum(X_valid_normalized/3,axis=3,keepdims=True)

image_shape = X_train_gray[0].shape
print(image_shape)

(32, 32, 1)


## Training
### Network Architecture

### Prepare the model

In [25]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(43,activation='sigmoid'))


### Execute the training
We stop when accuracy is over 0.95

In [26]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)             

In [30]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_gray_norm, y_train, epochs=20,batch_size=128, shuffle=True,
                    validation_data=(X_valid_gray_norm, y_valid))


Epoch 1/20
272/272 [==============================] - 1s 5ms/step - loss: 2.8597 - accuracy: 0.9024 - val_loss: 2.8792 - val_accuracy: 0.9136
Epoch 2/20
272/272 [==============================] - 1s 4ms/step - loss: 2.8586 - accuracy: 0.9018 - val_loss: 2.8797 - val_accuracy: 0.8848
Epoch 3/20
272/272 [==============================] - 1s 4ms/step - loss: 2.8569 - accuracy: 0.9060 - val_loss: 2.8806 - val_accuracy: 0.9002
Epoch 4/20
272/272 [==============================] - 1s 4ms/step - loss: 2.8559 - accuracy: 0.9088 - val_loss: 2.8829 - val_accuracy: 0.9020
Epoch 5/20
272/272 [==============================] - 1s 4ms/step - loss: 2.8540 - accuracy: 0.9134 - val_loss: 2.8769 - val_accuracy: 0.9063
Epoch 6/20
272/272 [==============================] - 1s 4ms/step - loss: 2.8543 - accuracy: 0.9129 - val_loss: 2.8795 - val_accuracy: 0.9063
Epoch 7/20
272/272 [==============================] - 1s 4ms/step - loss: 2.8532 - accuracy: 0.9134 - val_loss: 2.8805 - val_accuracy: 0.9032
Epoch 

## Test the model

In [31]:
test_loss, test_acc = model.evaluate(X_test_gray_norm,  y_test)

print('\nTest accuracy:', test_acc)

395/395 [==============================] - 1s 2ms/step - loss: 2.8738 - accuracy: 0.9123

Test accuracy: 0.9122723937034607
